In [1]:
from src.llm_plan.environment import Environment

env = Environment("./environments/static/TripPlanner.json")
env.plan

[['traveler.pddl'], ['orchestrator.pddl']]

In [2]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)

In [3]:
# Plan
responses = planner.plan()

----- traveler->pddl -----

----- System Prompt -----
You are an expert in PDDL for travel planning. You always provide a PDDL domain and a PDDL problem file. Enclose the domain in <domain></domain> tags, and the problem in <problem></problem> tags.

----- Prompt -----
Your name is traveler. You are planning a 14-day trip in Europe. Public information: ['Direct flights exist between London and Bucharest.', 'Direct flights exist between Reykjavik and London.']. Your personal constraints: ['I want to visit 3 European cities for 14 days.', 'I want to stay in London for 4 days.', 'I want to stay in Bucharest for 5 days.', 'I want to meet a friend in Bucharest between day 10 and day 14.', 'I want to stay in Reykjavik for 7 days.', 'I only take direct flights.']. Goal: Plan a 14-day trip across London, Bucharest, and Reykjavik that respects the stay durations and constraints.. Think step by step and provide a PDDL domain and problem that encode the trip plan. Do not make assumptions beyond t

In [4]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_orchestrator"
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [5]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [6]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: NUMBER does not name a known token



In [7]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import HypervisorDeepThinkPDDL, HypervisorSyntaxPDDL

num_attempts = 10

for i in range(num_attempts):
    
    # General validator
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / "refined_sas_plan",
        BASE_FOLDER / "refined_domain.pddl",
        BASE_FOLDER / "refined_problem.pddl",
    ]

    with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / 'refined_domain.pddl'} \
    {BASE_FOLDER / 'refined_problem.pddl'} \
    {BASE_FOLDER / 'refined_sas_plan'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # Syntax validator
    print(f"Validation attempt {i+1}/{num_attempts}")
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
    }

    deep_think = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: NUMBER does not name a known token

Validation attempt 1/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: 15 does not name a known token

Validation attempt 2/10
The plan is valid.
[stdout] Checking plan: tmp/TripPlanner/refined_sas_plan
Plan executed successfully - checking goal
Success: Plan valid
Value: 4.000000


Validation attempt 3/10
The plan is valid.
[stdout] Checking plan: tmp/TripPlanner/refined_sas_plan
Plan executed successfully - checking goal
Success: Plan valid
Value: 4.000000


Validation attempt 4/10
The plan is valid.
[stdout] Checking plan: tmp/TripPlanner/refined_sas_plan
Plan executed successfully - checking goal
Success: Plan valid
Value: 7.000000


Validation attempt 5/10
The plan is valid.
[stdout] Checking plan: tmp/TripPlanner/refined_sas_plan
Pla

In [8]:
# Final plan run and validation
with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "refined_sas_plan",
    BASE_FOLDER / "refined_domain.pddl",
    BASE_FOLDER / "refined_problem.pddl",
]

with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'refined_domain.pddl'} \
{BASE_FOLDER / 'refined_problem.pddl'} \
{BASE_FOLDER / 'refined_sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: 1 does not name a known token

